In [2]:
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup

from tqdm import tqdm
import csv
import os
import requests
import time

import pandas as pd

In [3]:
FIRST_RUN = False
cotation_url = ['https://www.slickcharts.com/sp500']

In [4]:
def scrape_isin(urls):
    driver = webdriver.Chrome()
    for url in urls:
        driver.get(url)
        # cookies = driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]')
        # wait for loading and then click on the button
        cookies = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]')))
        cookies.click()
        
        try:
            # get the table
            table = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[4]/div[2]/div[1]/div/div/table')))
            # get the rows
            rows = table.find_elements(By.TAG_NAME, 'tr')
            symbols = [row.find_elements(By.TAG_NAME, 'td')[2].text for row in rows[1:]]
            # get the headers
            headers = rows[0].find_elements(By.TAG_NAME, 'th')
        
            # get the isin
        except NoSuchElementException:
            print("Table element not found.")
        
        #Save the list in a txt file
        with open('../data/symbols.txt', 'w') as f:
            for item in symbols:
                f.write(item + '\n')
                
        return symbols

In [5]:
if FIRST_RUN:
    symbols = scrape_isin(cotation_url)
else:
    with open('../data/symbols.txt', 'r') as f:
        symbols = f.readlines()
        symbols = [symbol.strip() for symbol in symbols]

In [6]:
symbols

['MSFT',
 'AAPL',
 'NVDA',
 'AMZN',
 'META',
 'GOOGL',
 'GOOG',
 'BRK.B',
 'LLY',
 'AVGO',
 'JPM',
 'XOM',
 'UNH',
 'V',
 'TSLA',
 'MA',
 'PG',
 'JNJ',
 'HD',
 'MRK',
 'COST',
 'ABBV',
 'CVX',
 'CRM',
 'NFLX',
 'WMT',
 'AMD',
 'BAC',
 'PEP',
 'KO',
 'LIN',
 'ADBE',
 'TMO',
 'DIS',
 'WFC',
 'ACN',
 'MCD',
 'CSCO',
 'ORCL',
 'ABT',
 'QCOM',
 'CAT',
 'INTU',
 'GE',
 'IBM',
 'VZ',
 'AMAT',
 'DHR',
 'CMCSA',
 'COP',
 'NOW',
 'INTC',
 'TXN',
 'UBER',
 'PFE',
 'AMGN',
 'PM',
 'UNP',
 'RTX',
 'SPGI',
 'LOW',
 'GS',
 'ISRG',
 'NEE',
 'MU',
 'HON',
 'AXP',
 'ETN',
 'PGR',
 'BKNG',
 'LRCX',
 'ELV',
 'NKE',
 'T',
 'SYK',
 'MS',
 'C',
 'SCHW',
 'TJX',
 'BLK',
 'MDT',
 'DE',
 'UPS',
 'VRTX',
 'CI',
 'ADP',
 'BSX',
 'CB',
 'PLD',
 'LMT',
 'BA',
 'SBUX',
 'MMC',
 'BMY',
 'REGN',
 'ADI',
 'MDLZ',
 'KLAC',
 'BX',
 'FI',
 'PANW',
 'CVS',
 'TMUS',
 'GILD',
 'SNPS',
 'AMT',
 'CDNS',
 'CMG',
 'EOG',
 'SO',
 'TGT',
 'WM',
 'CME',
 'ICE',
 'MPC',
 'SHW',
 'DUK',
 'MO',
 'SLB',
 'FCX',
 'CL',
 'ZTS',
 'PH',
 '

In [7]:
len(symbols)

503

In [36]:
def scrape_financial(symbols, progress_file, mode):
    links = []
    driver = webdriver.Chrome()
    with open(progress_file, 'r') as f:
        progress = f.readlines()
        progress = [line.split(':')[0] for line in progress]

    symbols = [symbol for symbol in symbols if symbol not in progress]
    for symbol in symbols:
        url = f'https://fr.finance.yahoo.com/quote/{symbol}/history'
        driver.get(url)
        driver.delete_all_cookies()
        
        try:
            try:
                cookies = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="consent-page"]/div/div/div/form/div[2]/div[2]/button[2]')))
                cookies.click()
            except :
                pass
            
            date = driver.find_element(By.XPATH, '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[1]/div/div/div')
            date.click()
            
            date_select = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[1]/div[1]/div[1]/div/div/div[2]/div/ul[2]/li[4]/button/span')
            date_select.click()
            
            confirm = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[1]/div[1]/button')
            confirm.click()
            
            try : 
                link = driver.find_element(By.XPATH, '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[2]/span[2]/a')
                download_link = link.get_attribute('href')
                
                links.append(download_link)
                with open (progress_file, 'a') as f:
                    f.write(symbol + ':' + download_link + '\n')
            except :
                pass
        except:
            #Quit driver
            driver.quit()
            scrape_financial(symbols, progress_file='../data/links.txt')
        
    return links
    
links = scrape_financial(symbols, progress_file='../data/links.txt')

503 symbols to scrape
Mode not found
Mode not found
Mode not found
Mode not found
Mode not found
Mode not found
Mode select not found
Mode not found


: 

: 

In [23]:
len(links)

0

In [6]:
def download_data(file, download_path='../data/financials/'):
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    
    with open(file, 'r') as f:
        lines = f.readlines()
        urls = [(line.split(':')[0], line.split(':')[1] + ':' + line.split(':')[2]) for line in tqdm(lines, desc='Extracting URLs')]
    
    for symbol, url in tqdm(urls, desc='Downloading data'):
        try:
            r = requests.get(url, headers=header)
            with open(f'{download_path}{symbol}.csv', 'wb') as f:
                f.write(r.content)
        except:
            print(f'Error downloading {symbol}.')
        # time.sleep(5)
    return 'Files downloaded successfully.'
    
download_data('../data/links.txt')

Extracting URLs: 100%|██████████| 503/503 [00:00<?, ?it/s]


[('MSFT',
  'https://query1.finance.yahoo.com/v7/finance/download/MSFT?period1=511056000&period2=1713398400&interval=1d&events=history&includeAdjustedClose=true\n'),
 ('AAPL',
  'https://query1.finance.yahoo.com/v7/finance/download/AAPL?period1=345427200&period2=1713398400&interval=1d&events=history&includeAdjustedClose=true\n'),
 ('NVDA',
  'https://query1.finance.yahoo.com/v7/finance/download/NVDA?period1=916963200&period2=1713398400&interval=1d&events=history&includeAdjustedClose=true\n'),
 ('AMZN',
  'https://query1.finance.yahoo.com/v7/finance/download/AMZN?period1=863654400&period2=1713398400&interval=1d&events=history&includeAdjustedClose=true\n'),
 ('META',
  'https://query1.finance.yahoo.com/v7/finance/download/META?period1=1337299200&period2=1713398400&interval=1d&events=history&includeAdjustedClose=true\n'),
 ('GOOGL',
  'https://query1.finance.yahoo.com/v7/finance/download/GOOGL?period1=1092873600&period2=1713398400&interval=1d&events=history&includeAdjustedClose=true\n'),
